# Applying the A360 Photometry Mask

Ben Levine\
LSST Science Piplines version: Weekly 2025_28\
Container Size: large

This notebook demonstrates how to apply the current version of the A360 photometry mask to a galaxy catalog.

Contributors to the mask were Ben Levine, Anja von der Linden, Rance Solomon, and Yuanyuan Zhang.

Code for loading the data provided by "Shear profile around A360 using ComCam HSM shapes" by Combet, Englert, Fu, dell'Antonio, Adari.

In [ ]:
# general python packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
from lsst.daf.butler import Butler
import lsst.geom as geom
import lsst.afw.geom as afwGeom

In [ ]:
repo = '/repo/dp1'
collection = 'LSSTComCam/runs/DRP/DP1/v29_0_0/DM-50260'

butler = Butler(repo, collections=collection)
version_str = collection.split('/')
version = version_str[-2:][0]+'_'+version_str[-2:][1]
print(version)

skymap = butler.get('skyMap', skymap='lsst_cells_v1')

## Find tracts and patches for Abell 360 and load the catalogs

Find all the tracts/patches that falls in a given region around the A360 BCG, and store the results in a dictionary `tp_dict`

In [ ]:
# Position of the BCG for A360
ra_bcg = 37.862
dec_bcg = 6.98

# Looking for all patches in delta deg region around it
delta = 0.5
center = geom.SpherePoint(ra_bcg, dec_bcg, geom.degrees)
ra_min, ra_max = ra_bcg - delta, ra_bcg + delta
dec_min, dec_max = dec_bcg - delta, dec_bcg + delta

ra_range = (ra_min, ra_max)
dec_range = (dec_min, dec_max)
radec = [geom.SpherePoint(ra_range[0], dec_range[0], geom.degrees),
         geom.SpherePoint(ra_range[0], dec_range[1], geom.degrees),
         geom.SpherePoint(ra_range[1], dec_range[0], geom.degrees),
         geom.SpherePoint(ra_range[1], dec_range[1], geom.degrees)]

tracts_and_patches = skymap.findTractPatchList(radec)

tp_dict = {}
for tract_num in np.arange(len(tracts_and_patches)):
    tract_info = tracts_and_patches[tract_num][0]
    tract_idx = tract_info.getId()
    # All the patches around the cluster
    patches = []
    for i,patch in enumerate(tracts_and_patches[tract_num][1]):
        patch_info = tracts_and_patches[tract_num][1][i]
        patch_idx = patch_info.sequential_index
        patches.append(patch_idx)
    tp_dict.update({tract_idx:patches})
#tp_dict
#print(tracts_and_patches)

Load the object catalogs for all these tracts/patches, make basic cuts, and store in a single merged catalog `merged_cat`.

In [ ]:
# Get the object catlaog of these patches
if 'v29' in version:
    datasetType = 'object_patch'
else:
    datasetType = 'objectTable'
 
#datasetType = 'deepCoadd_obj'
merged_cat = pd.DataFrame()

for tract in tp_dict.keys():
    print(f'Loading objects from tract {tract}, patches:{tp_dict[tract]}')
    for patch in tp_dict[tract]:
#        print(patch)
        dataId = {'tract': tract, 'patch' : patch ,'skymap':'lsst_cells_v1'}
        obj_cat = butler.get(datasetType, dataId=dataId)
        if datasetType == 'object_patch': # new naming convention, and obj_cat is now an astropy table. 
            obj_cat = obj_cat.to_pandas() # convert to pandas to leave the rest of the code unchanged
        pattern = r"^g_|^z_"  
        # Drop columns matching the pattern
        obj_cat = obj_cat.drop(columns=[col for col in obj_cat.columns if re.match(pattern, col)])

        filt = obj_cat['detect_isPrimary']==True
        filt &= obj_cat['r_cModel_flag']== False
        filt &= obj_cat['i_cModel_flag']== False
        filt &= obj_cat['r_cModelFlux']>0
        filt &= obj_cat['i_cModelFlux']>0
        filt &= obj_cat['refExtendedness'] > 0.5


        merged_cat = pd.concat([merged_cat, obj_cat[filt]], ignore_index=True)
        

# Apply healpix map

In [ ]:
import healpy as hp
mask_map = np.load('/home/b/bclevine/A360/A360_maskmap_4096_v1.hp')
nside = hp.npix2nside(len(mask_map))

In [ ]:
px_obs = hp.ang2pix(nside, merged_cat['i_ra'], merged_cat['i_dec'], lonlat=True, nest=False)
idx_keep = np.isin(px_obs, np.where(mask_map==0)[0])

In [ ]:
plt.scatter(merged_cat['i_ra'], merged_cat['i_dec'], marker='.', s=0.2)
plt.scatter(merged_cat['i_ra'][idx_keep], merged_cat['i_dec'][idx_keep], marker='.', s=0.2)
plt.xlabel('RA')
plt.ylabel('Dec')

In [ ]:
print(f'Objects kept: {np.count_nonzero(idx_keep)}')
print(f'Proportion: {np.count_nonzero(idx_keep)/len(merged_cat):.4f}')